In [1]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# Train vs Test

In [2]:
# df_train = pd.read_csv("./dataset/train_preprocessed_imputed_preprocessed.csv")
# df_target = df_train.pop("CO2 Emissions(g/km)")
# df_train

In [3]:
# df_test = pd.read_csv("./dataset/test_preprocessed.csv")
# df_test.rename(
#     columns={
#         "Fuel Consumption City": "Fuel Consumption City (L/100Km)",
#         "Fuel Consumption Hwy": "Fuel Consumption Hwy (L/100Km)",
#         "Fuel Consumption Comb": "Fuel Consumption Comb (L/100Km)",
#     },
#     inplace=True,
# )

# df_test

In [4]:
# df_all = pd.concat([df_train, df_test])
# df_all

In [5]:
# df_all.info()

# Impute

In [6]:
# df_fuelconsumption = df_all[
#     [
#         "Fuel Consumption City (L/100Km)",
#         "Fuel Consumption Hwy (L/100Km)",
#         "Fuel Consumption Comb (L/100Km)",
#         # "CO2 Emissions(g/km)",
#     ]
# ]

# df_enginecylinders = df_all[
#     # ["Engine Size(L)", "Cylinders", "CO2 Emissions(g/km)", "Make", "Vehicle Class"] # with category features
#     ["Engine Size(L)", "Cylinders"]
# ]

In [7]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp = IterativeImputer(random_state=42)
# imp.set_output(transform="pandas")
# imp.fit(df_fuelconsumption)
# df_fuelconsumption_imputed = imp.transform(df_fuelconsumption)

# imp = IterativeImputer(random_state=42)
# imp.set_output(transform="pandas")
# imp.fit(df_enginecylinders)
# df_enginecylinders_imputed = imp.transform(df_enginecylinders)

### add data to df_all_fix1

In [8]:
# # there were data < 0, so we need to replace it with 0

# df_numerical_imputed = pd.concat(
#     [df_fuelconsumption_imputed, df_enginecylinders_imputed], axis=1
# )
# df_numerical_imputed = df_numerical_imputed.clip(lower=0)

# df_numerical_imputed

In [9]:
# df_fix1 = df_all.drop(df_numerical_imputed.columns.tolist(), axis=1)
# df_fix1 = pd.concat([df_fix1, df_numerical_imputed], axis=1)
# df_fix1

In [10]:
# df_categorical_imputed = df_fix1[
#     [
#         "Make",
#         "Vehicle Class",
#         "Transmission",
#         "Fuel Type",
#         # "CO2 Emissions(g/km)",
#         "Engine Size(L)",
#         "Cylinders",
#     ]
# ]

# for col in df_categorical_imputed.columns:
#     if df_categorical_imputed[col].dtype == "object":
#         df_categorical_imputed[col] = df_categorical_imputed[col].astype("category")

In [11]:
# import miceforest as mf

# # might consider "UserWarning: [Transmission,Fuel Type] have very rare categories, it is a good idea to group these, or set the min_data_in_leaf parameter to prevent lightgbm from outputting 0.0 probabilities."

# kds = mf.ImputationKernel(
#     df_categorical_imputed, save_all_iterations=True, random_state=42
# )
# kds.mice(10)
# df_categorical_imputed = kds.complete_data()

# df_categorical_imputed

In [12]:
# df_categorical_impupted_fix = df_categorical_imputed.copy()
# df_categorical_impupted_fix = df_categorical_impupted_fix.drop(
#     [
#         # "CO2 Emissions(g/km)",
#         "Engine Size(L)",
#         "Cylinders",
#     ],
#     axis=1,
# )

In [13]:
# df_fix2 = df_fix1.drop(df_categorical_impupted_fix.columns.tolist(), axis=1)
# df_fix2 = pd.concat([df_fix2, df_categorical_impupted_fix], axis=1)
# df_fix2

In [14]:
# df_fix3 = df_fix2.copy()
# df_fix3.reset_index(drop=True, inplace=True)
# df_fix3

In [15]:
# df_train_fix = df_fix3.iloc[:df_train.shape[0], :]
# df_train_fix

In [16]:
# df_test_fix = df_fix3.iloc[df_train.shape[0] :, :]
# df_test_fix.reset_index(drop=True, inplace=True)
# df_test_fix

In [17]:
# df_train_fix = pd.concat([df_train_fix, df_target], axis=1)

In [18]:
# df_train_fix.to_csv("./dataset/train_fix.csv", index=False)
# df_test_fix.to_csv("./dataset/test_fix.csv", index=False)

In [19]:
# from dython.nominal import associations

# associations(df_train_fix)
# associations(df_test_fix)
# associations(df_fix3)
# print()

# After Imputation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
df_train_fix = pd.read_csv("./dataset/train_fix.csv")
df_test_fix = pd.read_csv("./dataset/test_fix.csv")
df_test_id = df_test_fix['Id']
df_target = df_train_fix.pop("CO2 Emissions(g/km)")

# Preprocessing

Numerical Value:
- Fuel Consumption City (L/100Km) -> log, sqrt, boxcox, inverse
- Fuel Consumption Hwy (L/100Km) -> log, sqrt, boxcox, inverse
- Fuel Consumption Comb (L/100Km) -> log, sqrt, boxcox, inverse
- Engine Size(L) -> log, sqrt, boxcox, inverse
- Cylinders -> log, sqrt, boxcox, inverse

Categorical Value:
- Make -> one-hot
- Vehicle Class -> binary
- Transmission -> binary 
- Fuel Type -> one-hot

In [3]:
df_train_fix_preprocessed = df_train_fix.copy()
df_test_fix_preprocessed = df_test_fix.copy()

In [4]:
df_train_fix_preprocessed

,Id,Fuel Consumption City (L/100Km),Fuel Consumption Hwy (L/100Km),Fuel Consumption Comb (L/100Km),Engine Size(L),Cylinders,Make,Vehicle Class,Transmission,Fuel Type
0,1,8.640000,6.140000,7.514791,3.500000,6.0,FOLD,PICKUP TRUCK - STANDARD,A6,X
1,2,27.270000,30.760000,28.840000,5.300000,8.0,CHEVO,PICKUP TRUCK - STANDARD,A6,E
2,3,1.700000,2.030000,1.848477,4.400000,6.0,BMV,SUBCOMPACT,M6,Z
3,4,15.337423,15.900000,15.590609,2.057097,4.0,KIO,SUV - SMALL,AS6,X
4,5,20.150000,6.000000,13.800000,3.000000,6.0,BARUSU,MINICOMPACT,AS6,Z
...,...,...,...,...,...,...,...,...,...,...
137136,137137,33.710000,7.698229,22.026432,5.023977,8.0,BMV,SUBCOMPACT,AS8,Z
137137,137138,6.489293,0.870000,3.961175,1.500000,4.0,GONDA,STATION WAGON - SMALL,M6,X
137138,137139,2.990000,8.380000,5.412550,1.800000,4.0,NIRRAN,MID-SIZE,AV,X
137139,137140,11.480000,0.221408,6.400000,1.800000,4.0,TOYOTI,COMPACT,AV,X


In [5]:
# from sklearn.preprocessing import PowerTransformer <- MIGHT USE THIS

column_to_transform = [
    "Fuel Consumption City (L/100Km)",
    "Fuel Consumption Hwy (L/100Km)",
    "Fuel Consumption Comb (L/100Km)",
    "Engine Size(L)",
    "Cylinders",
]

In [6]:
for col in column_to_transform:
    df_train_fix_preprocessed["T_log_"+col] = np.log(df_train_fix_preprocessed[col] + 1)
    df_test_fix_preprocessed["T_log_"+col] = np.log(df_test_fix_preprocessed[col] + 1)

In [7]:
for col in column_to_transform:
    df_train_fix_preprocessed["T_sqrt_"+col] = np.sqrt(df_train_fix_preprocessed[col])
    df_test_fix_preprocessed["T_sqrt_"+col] = np.sqrt(df_test_fix_preprocessed[col])

In [8]:
from scipy import stats

for col in column_to_transform:
    df_train_fix_preprocessed["T_boxcox_"+col], _ = stats.boxcox(
        df_train_fix_preprocessed[col] + 1
    )
    df_test_fix_preprocessed["T_boxcox_"+col], _ = stats.boxcox(
        df_test_fix_preprocessed[col] + 1
    )

In [9]:
for col in column_to_transform:
    df_train_fix_preprocessed["T_inverse_"+col] = 1 / (df_train_fix_preprocessed[col] + 1)
    df_test_fix_preprocessed["T_inverse_"+col] = 1 / (df_test_fix_preprocessed[col] + 1)

In [10]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

categorical_features_onehot = [
    "Make",
    "Fuel Type",
]

categorical_features_binary = [
    "Vehicle Class",
    "Transmission",
]

df_train_categorical_one_hot = df_train_fix_preprocessed[
    categorical_features_onehot
].copy()
df_train_categorical_binary = df_train_fix_preprocessed[
    categorical_features_binary
].copy()

df_test_categorical_one_hot = df_test_fix_preprocessed[
    categorical_features_onehot
].copy()
df_test_categorical_binary = df_test_fix_preprocessed[
    categorical_features_binary
].copy()


encoder_onehot = OneHotEncoder(sparse_output=False)
encoder_onehot.set_output(transform="pandas")
train_onehot_encoded_data = encoder_onehot.fit_transform(df_train_categorical_one_hot)

# do the same with test data
test_onehot_encoded_data = encoder_onehot.transform(df_test_categorical_one_hot)

encoder_binary = BinaryEncoder(cols=categorical_features_binary)
encoder_binary.set_output(transform="pandas")
train_df_binary = encoder_binary.fit_transform(df_train_categorical_binary)

# do the same with test data
test_df_binary = encoder_binary.transform(df_test_categorical_binary)


onehot_encoded_data_cat_train = (
    train_onehot_encoded_data.copy().astype("int64").astype("category")
)
df_binary_cat_train = train_df_binary.copy().astype("int64").astype("category")

onehot_encoded_data_cat_test = (
    test_onehot_encoded_data.copy().astype("int64").astype("category")
)
df_binary_cat_test = test_df_binary.copy().astype("int64").astype("category")

In [11]:
df_train_log = df_train_fix_preprocessed.filter(regex="T_log_")
df_train_sqrt = df_train_fix_preprocessed.filter(regex="T_sqrt_")
df_train_boxcox = df_train_fix_preprocessed.filter(regex="T_boxcox_")
df_train_inverse = df_train_fix_preprocessed.filter(regex="T_inverse_")

df_train_to_model_log = pd.concat(
    [df_train_log, df_binary_cat_train, onehot_encoded_data_cat_train], axis=1
)
df_train_to_model_sqrt = pd.concat(
    [df_train_sqrt, df_binary_cat_train, onehot_encoded_data_cat_train], axis=1
)
df_train_to_model_boxcox = pd.concat(
    [df_train_boxcox, df_binary_cat_train, onehot_encoded_data_cat_train], axis=1
)
df_train_to_model_inverse = pd.concat(
    [df_train_inverse, df_binary_cat_train, onehot_encoded_data_cat_train], axis=1
)

In [12]:
df_test_log = df_test_fix_preprocessed.filter(regex="T_log_")
df_test_sqrt = df_test_fix_preprocessed.filter(regex="T_sqrt_")
df_test_boxcox = df_test_fix_preprocessed.filter(regex="T_boxcox_")
df_test_inverse = df_test_fix_preprocessed.filter(regex="T_inverse_")

df_test_to_model_log = pd.concat(
    [df_test_log, df_binary_cat_test, onehot_encoded_data_cat_test], axis=1
)
df_test_to_model_sqrt = pd.concat(
    [df_test_sqrt, df_binary_cat_test, onehot_encoded_data_cat_test], axis=1
)
df_test_to_model_boxcox = pd.concat(
    [df_test_boxcox, df_binary_cat_test, onehot_encoded_data_cat_test], axis=1
)
df_test_to_model_inverse = pd.concat(
    [df_test_inverse, df_binary_cat_test, onehot_encoded_data_cat_test], axis=1
)

In [13]:
df_test_to_model_log

,T_log_Fuel Consumption City (L/100Km),T_log_Fuel Consumption Hwy (L/100Km),T_log_Fuel Consumption Comb (L/100Km),T_log_Engine Size(L),T_log_Cylinders,Vehicle Class_0,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,...,Make_MITSU,Make_NIRRAN,Make_RYUNDAI,Make_TOLVO,Make_TOYOTI,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
0,2.810908,1.018847,2.341186,1.526056,1.949305,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,3.096030,2.733068,2.948807,1.547563,1.945910,0,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
2,3.238738,2.423984,2.950074,1.945910,2.291777,0,0,0,1,1,...,1,0,0,0,0,0,0,0,1,0
3,2.879198,2.827210,2.857619,1.722767,2.106460,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,3.752579,3.003267,3.481855,1.686399,2.197225,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58771,2.962943,3.086487,3.020425,1.458615,1.896902,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
58772,1.972671,2.028113,1.997891,0.955511,1.609438,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
58773,2.740372,2.485041,2.633605,1.504077,1.932141,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
58774,2.243002,1.721701,2.041248,1.791759,2.197225,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [14]:
df_test_to_model_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58776 entries, 0 to 58775
Data columns (total 41 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   T_log_Fuel Consumption City (L/100Km)  58776 non-null  float64 
 1   T_log_Fuel Consumption Hwy (L/100Km)   58776 non-null  float64 
 2   T_log_Fuel Consumption Comb (L/100Km)  58776 non-null  float64 
 3   T_log_Engine Size(L)                   58776 non-null  float64 
 4   T_log_Cylinders                        58776 non-null  float64 
 5   Vehicle Class_0                        58776 non-null  category
 6   Vehicle Class_1                        58776 non-null  category
 7   Vehicle Class_2                        58776 non-null  category
 8   Vehicle Class_3                        58776 non-null  category
 9   Vehicle Class_4                        58776 non-null  category
 10  Transmission_0                         58776 non-null  cat

# Modeling

- XGBoost
- LGBM
- DNN (NOT YET)
- ENSEMBLE

In [15]:
random_state = 42

In [16]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(random_state=random_state)

In [17]:
import lightgbm as lgb

model_lgbm = lgb.LGBMRegressor(random_state=random_state)

In [18]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Assuming you have training data X_train, y_train
# model_dnn = Sequential()
# model_dnn.add(
#     Dense(256, input_dim=df_train_to_model_log.shape[1], activation="relu")
# )  # Input layer
# model_dnn.add(Dense(256, activation="relu"))  # Hidden layer 1
# model_dnn.add(Dense(256, activation="relu"))  # Hidden layer 2
# model_dnn.add(Dense(1))  # Output layer

# model_dnn.compile(loss="mean_squared_error", optimizer="adam")

In [19]:
from sklearn.ensemble import StackingRegressor

model_ensemble = StackingRegressor(estimators=[("xgb", model_xgb), ("lgbm", model_lgbm)])

# try 2

In [20]:
df_test_to_model_boxcox

,T_boxcox_Fuel Consumption City (L/100Km),T_boxcox_Fuel Consumption Hwy (L/100Km),T_boxcox_Fuel Consumption Comb (L/100Km),T_boxcox_Engine Size(L),T_boxcox_Cylinders,Vehicle Class_0,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,...,Make_MITSU,Make_NIRRAN,Make_RYUNDAI,Make_TOLVO,Make_TOYOTI,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
0,5.244276,1.224908,3.947286,0.875550,0.517945,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,6.189469,4.587420,5.764516,0.881737,0.517858,0,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
2,6.705217,3.822731,5.768798,0.978827,0.524394,0,0,0,1,1,...,1,0,0,0,0,0,0,0,1,0
3,5.460810,4.837306,5.461867,0.928319,0.521420,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,8.830260,5.327378,7.781140,0.919186,0.523006,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58771,5.734662,5.569789,6.010334,0.855433,0.516538,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
58772,3.022169,2.957070,3.105215,0.665022,0.505789,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
58773,5.026833,3.967286,4.765046,0.869114,0.517499,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
58774,3.658092,2.364939,3.205094,0.944922,0.523006,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [24]:
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

# Assuming 'models' is a list of your models
models = [model_lgbm, model_xgb, model_ensemble]

# Create a list of your datasets
datasets = [
    df_train_to_model_log,
    df_train_to_model_sqrt,
    df_train_to_model_boxcox,
    df_train_to_model_inverse,
]

y = df_target

# Initialize a dictionary to hold your results
results = {}

# Create a KFold object
kf = KFold(n_splits=5)

# Loop over your models
for model in models:
    model_name = type(model).__name__
    results[model_name] = {}

    # Loop over your datasets
    for i, dataset in enumerate(datasets):
        results[model_name][f"dataset_{i+1}"] = {
            "rmse": [],
            "training_time": [],
        }

        # Perform cross-validation
        for train_index, test_index in kf.split(dataset):
            X_train, X_test = dataset.iloc[train_index], dataset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Create a StandardScaler object
            # scaler = StandardScaler()
            scaler = MinMaxScaler()

            numerical_column = X_train.select_dtypes(include=np.number).columns.tolist()
            non_numerical_columns = [
                col for col in X_train.columns if col not in numerical_column
            ]
            for col in non_numerical_columns:
                X_train.loc[:, col] = X_train.loc[:, col].astype("int64")
                X_test.loc[:, col] = X_test.loc[:, col].astype("int64")

            # Normalize the data
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Start the timer
            start_time = time.time()

            # Train the model
            model.fit(X_train, y_train)

            # End the timer
            end_time = time.time()

            # Calculate the training time
            training_time = end_time - start_time

            # Make predictions
            predictions = model.predict(X_test)

            # Evaluate the model
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)

            # Store the results
            results[model_name][f"dataset_{i+1}"]["rmse"].append(rmse)
            results[model_name][f"dataset_{i+1}"]["training_time"].append(training_time)

        # Calculate the mean RMSE and training time
        results[model_name][f"dataset_{i+1}"]["rmse"] = np.mean(
            results[model_name][f"dataset_{i+1}"]["rmse"]
        )
        results[model_name][f"dataset_{i+1}"]["training_time"] = np.mean(
            results[model_name][f"dataset_{i+1}"]["training_time"]
        )

        # Print the process
        print(
            f"Model: {model_name}, Dataset: {i+1}, Mean RMSE: {results[model_name][f'dataset_{i+1}']['rmse']:.4f}, Mean Training time: {results[model_name][f'dataset_{i+1}']['training_time']:.4f} seconds"
        )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005210 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1094
[LightGBM] [Info] Number of data points in the train set: 109712, number of used features: 41
[LightGBM] [Info] Start training from score 250.363262
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1094
[LightGBM] [Info] Number of data points in the train set: 109713, number of used features: 41
[LightGBM] [Info] Start training from score 250.481018
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005000 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [25]:
# Convert the nested dictionary into a pandas DataFrame
df_results = pd.concat({k: pd.DataFrame(v).T for k, v in results.items()}, axis=0)

# Reset the index and rename the columns for a cleaner look
df_results.reset_index(inplace=True)
df_results.columns = ["Model", "Dataset", "RMSE", "Training Time"]

df_results.to_csv("results_to_submit.csv")
df_sorted = df_results.sort_values(by="RMSE", ascending=True)
df_sorted

,Model,Dataset,RMSE,Training Time
10,StackingRegressor,dataset_3,46.239974,10.300298
9,StackingRegressor,dataset_2,46.249831,10.600114
8,StackingRegressor,dataset_1,46.268106,9.866058
11,StackingRegressor,dataset_4,46.298292,9.404344
5,XGBRegressor,dataset_2,46.566862,0.925737
6,XGBRegressor,dataset_3,46.566862,1.008521
4,XGBRegressor,dataset_1,46.598451,1.184074
7,XGBRegressor,dataset_4,46.665463,1.020070
2,LGBMRegressor,dataset_3,46.920581,0.594900
3,LGBMRegressor,dataset_4,46.941424,0.853548


In [32]:
import time
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Assuming 'models' is a list of your models
models = [model_lgbm, model_xgb, model_ensemble]

# Create a list of your train datasets
train_datasets = [
    df_train_to_model_log,
    df_train_to_model_sqrt,
    df_train_to_model_boxcox,
    df_train_to_model_inverse,
]

# Create a list of your test datasets
test_datasets = [
    df_test_to_model_log,
    df_test_to_model_sqrt,
    df_test_to_model_boxcox,
    df_test_to_model_inverse,
]

y = df_target

# Initialize a dictionary to hold your results
results = {}

# Loop over your models
for model in models:
    model_name = type(model).__name__
    results[model_name] = {}

    # Loop over your datasets
    for i, (train_dataset, test_dataset) in enumerate(
        zip(train_datasets, test_datasets)
    ):
        results[model_name][f"dataset_{i+1}"] = {
            "predictions": [],
            "training_time": [],
        }

        X_train = train_dataset
        y_train = y

        # Create a MinMaxScaler object
        scaler = MinMaxScaler()

        numerical_column = X_train.select_dtypes(include=np.number).columns.tolist()
        non_numerical_columns = [
            col for col in X_train.columns if col not in numerical_column
        ]
        for col in non_numerical_columns:
            X_train.loc[:, col] = X_train.loc[:, col].astype("int64")

        # Normalize the train data
        X_train = scaler.fit_transform(X_train)

        # Start the timer
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train)

        # End the timer
        end_time = time.time()

        # Calculate the training time
        training_time = end_time - start_time

        # Normalize the test data
        X_test = test_dataset
        for col in non_numerical_columns:
            X_test.loc[:, col] = X_test.loc[:, col].astype("int64")
        X_test = scaler.transform(X_test)

        # Make predictions
        predictions = model.predict(X_test)

        # Store the results
        results[model_name][f"dataset_{i+1}"]["predictions"] = predictions
        results[model_name][f"dataset_{i+1}"]["training_time"] = training_time

        # Print the process
        print(
            f"Model: {model_name}, Dataset: {i+1}, Training time: {results[model_name][f'dataset_{i+1}']['training_time']:.4f} seconds"
        )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1094
[LightGBM] [Info] Number of data points in the train set: 137141, number of used features: 41
[LightGBM] [Info] Start training from score 250.471627
Model: LGBMRegressor, Dataset: 1, Training time: 0.9939 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1094
[LightGBM] [Info] Number of data points in the train set: 137141, number of used features: 41
[LightGBM] [Info] Start training from score 250.471627
Model: LGBMRegressor, Dataset: 2, Training time: 0.7490 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threadi

In [33]:
# Flatten the results dictionary
flat_results = []
for model_name, datasets in results.items():
    for dataset_name, metrics in datasets.items():
        flat_results.append(
            {
                "model_name": model_name,
                "dataset_name": dataset_name,
                "predictions": metrics["predictions"],
                "training_time": metrics["training_time"],
            }
        )

# Convert the flattened results to a DataFrame
df_results_submit = pd.DataFrame(flat_results)
df_results_submit

,model_name,dataset_name,predictions,training_time
0,LGBMRegressor,dataset_1,"[257.4333843577355, 391.2711779732619, 235.414...",0.993876
1,LGBMRegressor,dataset_2,"[257.4333843577355, 391.2711779732619, 235.414...",0.748997
2,LGBMRegressor,dataset_3,"[240.04145429792575, 373.92650560779424, 333.4...",1.099655
3,LGBMRegressor,dataset_4,"[255.3775955289704, 390.4991780239178, 236.479...",1.364389
4,XGBRegressor,dataset_1,"[253.64839, 397.09702, 247.60767, 309.2603, 36...",1.742997
5,XGBRegressor,dataset_2,"[253.64839, 397.09702, 247.60767, 309.2603, 36...",1.933057
6,XGBRegressor,dataset_3,"[173.94583, 304.45428, 243.39618, 321.24396, 4...",1.896135
7,XGBRegressor,dataset_4,"[257.9846, 396.9588, 218.35283, 310.51492, 358...",1.954471
8,StackingRegressor,dataset_1,"[255.44774377629415, 399.7096292400326, 242.08...",12.761528
9,StackingRegressor,dataset_2,"[255.44783628527404, 399.73423370733474, 242.0...",10.846962


In [31]:
stacking_regressor_predictions = results["StackingRegressor"]["dataset_3"][
    "predictions"
]
stacking_regressor_predictions

array([200.98841461, 337.69175784, 283.71207647, ..., 236.42852584,
       277.49618488, 223.50738832])

In [38]:
df_sample_submission = pd.read_csv("./dataset/sample_submission.csv")

In [39]:
df_sample_submission["CO2 Emissions(g/km)"] = stacking_regressor_predictions
df_sample_submission

,Id,CO2 Emissions(g/km)
0,137142,200.988415
1,137143,337.691758
2,137144,283.712076
3,137145,310.094764
4,137146,390.320249
...,...,...
58771,195913,249.387491
58772,195914,253.341301
58773,195915,236.428526
58774,195916,277.496185


In [ ]:
df_sample_submission.to_csv("stack_regressor.csv", index=False)